## PyTorch installation

pip install torch torchvision torchaudio

for more information https://pytorch.org/get-started/locally/

pip install numpy --upgrade

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
print(torch.__version__)

1.12.0+cpu


### (0) Preparing data 

In [3]:
# Loading the data set
T = np.loadtxt('heart.dat')

In [4]:
print(len(T[:, 13]))

270


In [5]:
# Extract the real features
X_numpy = T[:, [0, 3, 4, 7, 9, 11]]

In [6]:
# Extract and transform the markers
Y_numpy = T[:, 13] - 1
print(Y_numpy)

[1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1.
 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0.
 1. 0. 0. 0. 0. 1.]


In [7]:
# from numpy array to tensor
X = torch.from_numpy(X_numpy.astype(np.float32))
print(X_numpy)
print(X)
print(X.dtype)

[[7.00e+01 1.30e+02 3.22e+02 1.09e+02 2.40e+00 3.00e+00]
 [6.70e+01 1.15e+02 5.64e+02 1.60e+02 1.60e+00 0.00e+00]
 [5.70e+01 1.24e+02 2.61e+02 1.41e+02 3.00e-01 0.00e+00]
 ...
 [5.60e+01 1.40e+02 2.94e+02 1.53e+02 1.30e+00 0.00e+00]
 [5.70e+01 1.40e+02 1.92e+02 1.48e+02 4.00e-01 0.00e+00]
 [6.70e+01 1.60e+02 2.86e+02 1.08e+02 1.50e+00 3.00e+00]]
tensor([[7.0000e+01, 1.3000e+02, 3.2200e+02, 1.0900e+02, 2.4000e+00, 3.0000e+00],
        [6.7000e+01, 1.1500e+02, 5.6400e+02, 1.6000e+02, 1.6000e+00, 0.0000e+00],
        [5.7000e+01, 1.2400e+02, 2.6100e+02, 1.4100e+02, 3.0000e-01, 0.0000e+00],
        ...,
        [5.6000e+01, 1.4000e+02, 2.9400e+02, 1.5300e+02, 1.3000e+00, 0.0000e+00],
        [5.7000e+01, 1.4000e+02, 1.9200e+02, 1.4800e+02, 4.0000e-01, 0.0000e+00],
        [6.7000e+01, 1.6000e+02, 2.8600e+02, 1.0800e+02, 1.5000e+00, 3.0000e+00]])
torch.float32


In [8]:
# from numpy array to tensor
Y = torch.from_numpy(Y_numpy.astype(np.float32))
# Y = Y.view(Y.shape[0], 1)
print(Y_numpy)
print(Y)

[1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1.
 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0.
 1. 0. 0. 0. 0. 1.]
tensor([1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
        1., 1., 0

In [9]:
# Number of data pairs and features 
m, d = X.shape
print(m, d)

270 6


### (1) Splitting our dataset into a train/test split.

Random selection of the indices of the training and test data

In [10]:
# Share of training data
p = 0.7
data_ind = np.random.permutation(m)
print((np.ceil(p*m)+1))
ind_train = data_ind[:int((np.ceil(p*m)+1))]
ind_test = [i for i in data_ind if i not in ind_train]

190.0


In [11]:
# Training data
X_train = X[ind_train, :]
Y_train = Y[ind_train]
print(X_train)
print(Y_train)

tensor([[ 51., 140., 261., 186.,   0.,   0.],
        [ 49., 130., 269., 163.,   0.,   0.],
        [ 41., 126., 306., 163.,   0.,   0.],
        ...,
        [ 47., 108., 243., 152.,   0.,   0.],
        [ 58., 100., 248., 122.,   1.,   0.],
        [ 61., 130., 330., 169.,   0.,   0.]])
tensor([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
        1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0.,
        0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
        0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0.,
        0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
        1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1.,
        1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.

In [12]:
# Test data
X_test = X[ind_test, :]
Y_test = Y[ind_test]
print(X_test)
print(Y_test)

tensor([[6.4000e+01, 1.2000e+02, 2.4600e+02, 9.6000e+01, 2.2000e+00, 1.0000e+00],
        [6.4000e+01, 1.2800e+02, 2.6300e+02, 1.0500e+02, 2.0000e-01, 1.0000e+00],
        [5.1000e+01, 1.0000e+02, 2.2200e+02, 1.4300e+02, 1.2000e+00, 0.0000e+00],
        [6.2000e+01, 1.2400e+02, 2.0900e+02, 1.6300e+02, 0.0000e+00, 0.0000e+00],
        [5.0000e+01, 1.4400e+02, 2.0000e+02, 1.2600e+02, 9.0000e-01, 0.0000e+00],
        [4.6000e+01, 1.3800e+02, 2.4300e+02, 1.5200e+02, 0.0000e+00, 0.0000e+00],
        [5.4000e+01, 1.6000e+02, 2.0100e+02, 1.6300e+02, 0.0000e+00, 1.0000e+00],
        [5.8000e+01, 1.0000e+02, 2.3400e+02, 1.5600e+02, 1.0000e-01, 1.0000e+00],
        [4.6000e+01, 1.0100e+02, 1.9700e+02, 1.5600e+02, 0.0000e+00, 0.0000e+00],
        [4.6000e+01, 1.4000e+02, 3.1100e+02, 1.2000e+02, 1.8000e+00, 2.0000e+00],
        [6.4000e+01, 1.3000e+02, 3.0300e+02, 1.2200e+02, 2.0000e+00, 2.0000e+00],
        [5.9000e+01, 1.7000e+02, 3.2600e+02, 1.4000e+02, 3.4000e+00, 0.0000e+00],
        [4.2000e

### (2) Building the PyTorch Model Class

In [13]:
class LogisticRegression(torch.nn.Module):
     def __init__(self, input_dim, output_dim):
         super(LogisticRegression, self).__init__()
         self.linear = torch.nn.Linear(input_dim, output_dim)
     def forward(self, x):
         outputs = torch.sigmoid(self.linear(x))
         return outputs

### (3) Initializing the Model

In [14]:
# assigning some hyper-parameters:
epochs = 1000 # Indicates the number of passes through the entire training dataset the network has completed
input_dim = d 
output_dim = 1 # Single output 
learning_rate = 0.01

<br>Epoch: Indicates the number of passes through the entire training dataset the network has completed
<br>
<br>Learning_rate: A tuning parameter in an optimization algorithm that determines the step size at each iteration while moving toward a minimum of a loss function
- High learning rate means you might never be able to reach a minimum.
- Low learning rate will take longer.

In [15]:
model = LogisticRegression(input_dim, output_dim)

### (4) Initializing the Loss Function and the Optimizer

##### SGD

In [16]:
# Binary Cross Entropy Loss
criterion_SGD = torch.nn.BCELoss() 

In [17]:
# SGD: Implements stochastic gradient descent (optionally with momentum)
optimizer_SGD = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [18]:
# adjusting learning rate
# ExponentialLR: decays the learning rate of each parameter group by gamma every epoch.
scheduler_SGD = torch.optim.lr_scheduler.ExponentialLR(optimizer_SGD, gamma=0.9)

##### Adam

In [19]:
# Binary Cross Entropy Loss
criterion_Adam = torch.nn.BCELoss() 

In [20]:
# Adam: Implements Adam algorithm
optimizer_Adam = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
scheduler_Adam = torch.optim.lr_scheduler.ExponentialLR(optimizer_Adam, gamma=0.9)

##### LBFGS

In [22]:
# Binary Cross Entropy Loss
criterion_LBFGS = torch.nn.BCELoss() 

In [23]:
# LBFGS: Implements L-BFGS algorithm, heavily inspired by minFunc
optimizer_LBFGS = torch.optim.LBFGS(model.parameters(), lr=learning_rate)

In [24]:
scheduler_LBFGS = torch.optim.lr_scheduler.ExponentialLR(optimizer_LBFGS, gamma=0.9)

### (5) Train the Model

In [25]:
print(X_train.shape)
print(Y_train.shape)

torch.Size([190, 6])
torch.Size([190])


In [26]:
def training_model(criterion, optimizer, scheduler):
    losses = []
    losses_test = []
    Iterations = []
    iter = 0

    for epoch in range(epochs):
    # for epoch in tqdm(range(int(epochs)),desc='Training Epochs'):
        x = X_train
        labels = Y_train
        def closure():
            optimizer.zero_grad() # Setting our stored gradients equal to zero
            outputs = model(X_train)
            loss = criterion(torch.squeeze(outputs), labels) 
            loss.backward() # Computes the gradient of the given tensor w.r.t. the weights/bias
            return loss

        optimizer.step(closure) # Updates weights and biases with the optimizer (SGD)
        scheduler.step()
        # print(scheduler.get_lr())
        # loss = closure()[0]
        # outputs = closure()[1]

        iter+=1
        if iter%1000==0:
            with torch.no_grad():
                # Calculating the loss and accuracy for the test dataset
                correct_test = 0
                total_test = 0
                print(Y_train)
                print(Y_test)
                
                outputs_test = torch.squeeze(model(X_test))
                loss_test = criterion(outputs_test, Y_test)
                print(f'model parameters: {list(model.parameters())}')

                predicted_test = outputs_test.round().detach().numpy()
                print(f'predictied test {predicted_test}')
                
                total_test += Y_test.size(0)
                correct_test += np.sum(predicted_test == Y_test.detach().numpy())
                accuracy_test = 100 * correct_test/total_test
                losses_test.append(loss_test.item())
                print(f"Iteration: {iter}. \nTest - Loss: {loss_test.item()}. Accuracy: {accuracy_test}")
                
                print(outputs_test.size())
                print(Y_test.size())
"""
                # Calculating the loss and accuracy for the train dataset
                total = 0
                correct = 0
                total += Y_train.size(0)
                correct += np.sum(torch.squeeze(outputs).round().detach().numpy() == Y_train.detach().numpy())
                accuracy = 100 * correct/total
                losses.append(loss.item())
                Iterations.append(iter)
"""
                # print(f"Iteration: {iter}. \nTest - Loss: {loss_test.item()}. Accuracy: {accuracy_test}")
                # print(f"Train -  Loss: {loss.item()}. Accuracy: {accuracy}\n")


'\n                # Calculating the loss and accuracy for the train dataset\n                total = 0\n                correct = 0\n                total += Y_train.size(0)\n                correct += np.sum(torch.squeeze(outputs).round().detach().numpy() == Y_train.detach().numpy())\n                accuracy = 100 * correct/total\n                losses.append(loss.item())\n                Iterations.append(iter)\n'

### (6) Test the Models

SGD

In [27]:
training_model(criterion_SGD, optimizer_SGD, scheduler_SGD)

tensor([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
        1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0.,
        0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
        0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0.,
        0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
        1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1.,
        1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
        1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0.,
        0., 0., 0., 1., 1., 1., 0., 1., 0., 1.])
tensor([1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0.,

Adam

In [28]:
# training_model(criterion_Adam, optimizer_Adam, scheduler_Adam)

LBFGS

In [29]:
# training_model(criterion_LBFGS, optimizer_LBFGS, scheduler_LBFGS)

### Adam

Learning rate 

https://pytorch.org/docs/stable/optim.html#torch.optim.Optimizer

Aufgaben:
- Log_Reg_script in Pytorch implementieren (2 dimenionaler Datensatz, es wird einfacher. Alles, was im Script steht, muss gleich in Pytorch implementiert werden, die Grafiken auch).
- Heart script verbessern, dafür muss ich die erlernte Gewichte von Matlab nehmen und hier vergleichen, das Resultat hier un in Matlab muss identisch sein.
- Wenn ich mit allem fertig bin, muss SVM in Pytorch auch implementieren.


Der nächste Termin ist am 22.08 um 10:00.